In [1]:
!pip install -U denoiser
!pip install audiosegment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 6.0 MB/s eta 0:00:00
  Created wheel for denoiser: filename=denoiser-0.1.5-py3-none-any.whl size=43822 sha256=efd993539e277f8d24580522731a0c2b8845d06759b3938834f52fe5df677546
  Stored in directory: /root/.cache/pip/wheels/f1/57/3f/7ce0ae586d457c598b2cb275a37801ff5a705f656ba7a0215e
  Created wheel for julius: filename=julius-0.2.7-py3-none-any.whl size=21870 sha256=493951844ee971402362d5e18e036470dc9d452b214e126124382e3eb832e410
  Stored in directory: /root/.cache/pip/wheels/b9/b2/05/f883527ffcb7f2ead5438a2c23439aa0c881eaa9a4c80256f4
Successfully built denoiser julius
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ...

In [2]:
from scipy.io import wavfile
from IPython import display as disp
import torch
import torchaudio
from denoiser import pretrained
from denoiser.dsp import convert_audio
from pathlib import Path
from pydub.silence import split_on_silence
import numpy as np
import audiosegment
from pydub import AudioSegment

In [3]:
import os
os.mkdir(Path("/kaggle/working/data"))  

In [4]:
model = pretrained.dns64().cuda()

Downloading: "https://dl.fbaipublicfiles.com/adiyoss/denoiser/dns64-a7761ff99a7d5bb6.th" to /root/.cache/torch/hub/checkpoints/dns64-a7761ff99a7d5bb6.th
100%|██████████| 128M/128M [00:00<00:00, 265MB/s]


In [5]:
def denose(file_path):
    wav, sr = torchaudio.load(file_path)
    wav = convert_audio(wav.cuda(), sr, model.sample_rate, model.chin)
    with torch.no_grad():
        denoised = model(wav[None])[0]
    
    file_path = file_path.split('/')[-1]
    
    wavfile.write(f'/kaggle/working/data/{file_path}', model.sample_rate, denoised.data.cpu().numpy().T)

In [6]:
def remove_silence(file_path):
    audio_format = "wav"
    
    sound = AudioSegment.from_file(file_path, format = audio_format)
    audio_chunks = split_on_silence(sound
                            ,min_silence_len = 100
                            ,silence_thresh = -30
                            ,keep_silence = 500
                        )

    combined = AudioSegment.empty()
    for chunk in audio_chunks:
        combined += chunk
        
    file_path = file_path.split('/')[-1]
    combined.export(f'/kaggle/working/data/{file_path}', format = audio_format)

In [7]:
p = Path("/kaggle/input/control-group")

for x in p.iterdir():
    denose(str(x)) 

new_p = Path('/kaggle/working/data')

for x in new_p.iterdir():
    remove_silence(str(x)) 

In [8]:
!zip -r control_denoised.zip "/kaggle/working/data"

  adding: kaggle/working/data/ (stored 0%)
  adding: kaggle/working/data/PN-026-instr-1-chair (mp3cut.net).wav (deflated 7%)
  adding: kaggle/working/data/PN-002-pic-1-sportsman (mp3cut.net).wav (deflated 8%)
  adding: kaggle/working/data/PN-051-pic-1-winterday (mp3cut.net).wav (deflated 9%)
  adding: kaggle/working/data/PN-051-pers-1-trip (mp3cut.net).wav (deflated 8%)
  adding: kaggle/working/data/PN-303-instr-1-chair (mp3cut.net).wav (deflated 7%)
  adding: kaggle/working/data/PN-003-pic-1-sportsman (mp3cut.net).wav (deflated 6%)
  adding: kaggle/working/data/PN-068-instr-1-bench (mp3cut.net).wav (deflated 8%)
  adding: kaggle/working/data/PN-057-pers-1-trip (mp3cut.net).wav (deflated 7%)
  adding: kaggle/working/data/PN-319-instr-1-table (mp3cut.net).wav (deflated 8%)
  adding: kaggle/working/data/PN-319-pic-1-party (mp3cut.net).wav (deflated 8%)
  adding: kaggle/working/data/PN-017-pic-1-adventure (mp3cut.net).wav (deflated 7%)
  adding: kaggle/working/data/PN-050-instr-1-bench (m